### Predicting the amount customers spend each year

In [1]:
#importing the necessary libraries
from pyspark.sql import SparkSession

#import the ml libraries
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#import the regression libraries
from pyspark.ml.regression import LinearRegression

In [2]:
#starting the spark session
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [3]:
#loading the dataset
all_data = spark.read.csv('09.Ecommerce_Customers.csv', inferSchema=True, header=True)

In [4]:
#printing the schema to explore the data
all_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
#explore a row object and print each column and its content
x = 0
for i in all_data.head(1)[0]:
    print (all_data.columns[x],' : ',i)
    x+=1  

Email  :  mstephenson@fernandez.com
Address  :  835 Frank TunnelWrightmouth, MI 82180-9605
Avatar  :  Violet
Avg Session Length  :  34.49726772511229
Time on App  :  12.65565114916675
Time on Website  :  39.57766801952616
Length of Membership  :  4.0826206329529615
Yearly Amount Spent  :  587.9510539684005


In [6]:
#create an vector assemble object
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')

In [7]:
#transform the data
output = assembler.transform(all_data)

In [8]:
#explore the features column in the  transformed data
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [9]:
#explore the transformed data row objects
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [10]:
#notice that features column has the data from the selected columns in the assembler
output.head(1)[0][8]

DenseVector([34.4973, 12.6557, 39.5777, 4.0826])

In [11]:
#save the selected features into a new data 
processed_data = output.select('features', 'Yearly Amount Spent')

In [12]:
#explore the new dataframe
processed_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [13]:
#split the data into train and test sets
train_data, test_data = processed_data.randomSplit([0.7,0.3])

In [14]:
#create a linear regression session
lin_reg = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='Yearly_Amount_Spent_Prediction')

In [15]:
#training the model
lin_reg_model = lin_reg.fit(train_data)

In [16]:
#explore the model
result_train = lin_reg_model.summary

In [17]:
#explore the predictions with the train data
result_train.predictions.show()

+--------------------+-------------------+------------------------------+
|            features|Yearly Amount Spent|Yearly_Amount_Spent_Prediction|
+--------------------+-------------------+------------------------------+
|[29.5324289670579...|  408.6403510726275|             397.9401436736673|
|[30.4925366965402...|  282.4712457199145|              287.741975054415|
|[30.5743636841713...| 442.06441375806565|            441.63500909287336|
|[30.7377203726281...|  461.7807421962299|            451.00855057437593|
|[30.8162006488763...|   266.086340948469|            283.22426892120984|
|[30.8364326747734...|  467.5019004269896|             471.6066137085063|
|[30.9716756438877...|  494.6386097568927|            487.54894188863886|
|[31.0472221394875...|  392.4973991890214|            387.77227348511724|
|[31.0662181616375...| 448.93329320767435|             461.6138636379503|
|[31.1695067987115...|  427.3565308022928|             417.3212080670396|
|[31.2606468698795...|  421.3266312569

## Evaluate the model with the test data

In [18]:
#introduce the the test data
result_test = lin_reg_model.evaluate(test_data)

In [19]:
#Explore the predictions with the test _data
result_test.predictions.show()

+--------------------+-------------------+------------------------------+
|            features|Yearly Amount Spent|Yearly_Amount_Spent_Prediction|
+--------------------+-------------------+------------------------------+
|[30.3931845423455...|  319.9288698031936|            331.62608209151176|
|[30.8794843441274...|  490.2065999848547|            493.85159988363284|
|[31.0613251567161...|  487.5554580579016|            493.44170600242273|
|[31.1239743499119...|  486.9470538397658|             508.1894655725175|
|[31.1280900496166...|  557.2526867470547|             564.3217595060078|
|[31.2834474760581...|  591.7810894256675|             569.2926807770557|
|[31.3662121671876...|  430.5888825564849|             426.5657648763731|
|[31.3895854806643...|  410.0696110599829|             409.2576675553214|
|[31.4474464941278...|   418.602742095224|             425.8888646770447|
|[31.5316044825729...| 436.51560572936256|             432.8076991154285|
|[31.5741380228732...|  544.4092721605

## Explore the evaluation metrics 

In [20]:
#MAE: mean absolute error 
print(result_test.meanAbsoluteError)

7.739817014633166


In [21]:
#MSE: mean squared error
print(result_test.meanSquaredError)

100.53637605723358


In [22]:
#RMSE: root mean squared error
print(result_test.rootMeanSquaredError)

10.026782936577094


In [23]:
#R squared
print(result_test.r2)

0.9861756166731432


In [24]:
#explore the basis stats of the processed data
processed_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## Deploying the model

In [25]:
#create an unlabeled data set using the test data
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2834474760581...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.4474464941278...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8293464559211...|
|[31.8530748017465...|
+--------------------+
only showing top 20 rows



In [26]:
#deploy the model
result_unlabeled = lin_reg_model.transform(unlabeled_data)

In [27]:
#explore the predictions
result_unlabeled.show()

+--------------------+------------------------------+
|            features|Yearly_Amount_Spent_Prediction|
+--------------------+------------------------------+
|[30.3931845423455...|            331.62608209151176|
|[30.8794843441274...|            493.85159988363284|
|[31.0613251567161...|            493.44170600242273|
|[31.1239743499119...|             508.1894655725175|
|[31.1280900496166...|             564.3217595060078|
|[31.2834474760581...|             569.2926807770557|
|[31.3662121671876...|             426.5657648763731|
|[31.3895854806643...|             409.2576675553214|
|[31.4474464941278...|             425.8888646770447|
|[31.5316044825729...|             432.8076991154285|
|[31.5741380228732...|             558.4084881993199|
|[31.6005122003032...|             460.8841700102596|
|[31.6098395733896...|            427.42082847421807|
|[31.6253601348306...|            381.02046440739014|
|[31.7366356860502...|            494.13619194425723|
|[31.8124825597242...|      

In [28]:
#individual inquiry
lin_reg_model.transform(unlabeled_data)

DataFrame[features: vector, Yearly_Amount_Spent_Prediction: double]